In [1]:
import keras
import tensorflow
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.models import Sequential,Model
from keras.layers import InputLayer,Dense,Embedding,SimpleRNN,LSTM,Bidirectional,Masking,TimeDistributed
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import brown,treebank
from tensorflow.keras.preprocessing.text import Tokenizer
### Perform text preprocessing

In [2]:
brown

<CategorizedTaggedCorpusReader in 'C:\\Users\\DELL\\AppData\\Roaming\\nltk_data\\corpora\\brown'>

In [3]:
#nltk.download('brown')

In [4]:
brown.sents()

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

In [5]:
len(brown.sents())

57340

In [6]:
" ".join(brown.sents()[0])

"The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place ."

In [7]:
treebank

<BracketParseCorpusReader in 'C:\\Users\\DELL\\AppData\\Roaming\\nltk_data\\corpora\\treebank\\combined'>

In [8]:
#nltk.download('treebank')

In [9]:
treebank.sents()

[['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.'], ['Mr.', 'Vinken', 'is', 'chairman', 'of', 'Elsevier', 'N.V.', ',', 'the', 'Dutch', 'publishing', 'group', '.'], ...]

In [10]:
len(treebank.sents())

3914

In [11]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [12]:
d2=brown.tagged_sents(tagset='universal')

In [13]:
d1=treebank.tagged_sents(tagset='universal')

In [14]:
dataset=d1+d2

In [15]:
len(dataset)

61254

In [16]:
x=[]
y=[]
for l in dataset:
    x1=[]
    y1=[]
    for t in l:
        x1.append(t[0])
        y1.append(t[1])
        
    x.append(x1)
    y.append(y1)


In [17]:
len(x)

61254

In [18]:
len(y)

61254

In [19]:
len(x[0])==len(y[0])

True

In [20]:
#x

In [21]:
#y

In [22]:
c=0
for i in range(len(x)):
    if len(x[i])==len(y[i]):
        pass
    else:
        c+=1

In [23]:
c

0

In [24]:
## Finding out vocabulary
voc=[]
for q in x:
    for word in q:
        if word.lower() not in voc:
            voc.append(word.lower())

## Finding out tags
tags=[]
for tag_list in y:
    for tag in tag_list:
        if tag.lower() not in voc:
            tags.append(tag.lower())

In [25]:
tk_x=Tokenizer()
tk_x.fit_on_texts(x)

tk_y=Tokenizer()
tk_y.fit_on_texts(y)

In [26]:
len(tk_x.word_index)

53232

In [27]:
len(tk_y.word_index)

12

In [28]:
tk_y.word_index

{'noun': 1,
 'verb': 2,
 '.': 3,
 'adp': 4,
 'det': 5,
 'adj': 6,
 'adv': 7,
 'pron': 8,
 'conj': 9,
 'prt': 10,
 'num': 11,
 'x': 12}

In [29]:
x_d=tk_x.texts_to_sequences(x)
y_d=tk_y.texts_to_sequences(y)

In [30]:
len(x_d)

61254

In [31]:
len(y_d)

61254

In [32]:
from keras.utils import pad_sequences

In [33]:
final_x_d = pad_sequences(x_d,padding='post')
final_y_d = pad_sequences(y_d,padding='post')

In [34]:
final_x_d.shape

(61254, 271)

In [35]:
final_y_d.shape

(61254, 271)

In [36]:
final_y_d

array([[1, 1, 3, ..., 0, 0, 0],
       [1, 1, 2, ..., 0, 0, 0],
       [1, 1, 3, ..., 0, 0, 0],
       ...,
       [5, 1, 4, ..., 0, 0, 0],
       [8, 2, 5, ..., 0, 0, 0],
       [4, 5, 8, ..., 0, 0, 0]])

In [37]:
from keras.utils import to_categorical

In [38]:
final_y_d = to_categorical(final_y_d)

In [39]:
model=Sequential()
model.add(InputLayer(shape=(271,)))
model.add(Embedding(input_dim=len(tk_x.word_index)+1,output_dim=5)) #Normal Embedding
model.add(SimpleRNN(units=50,return_sequences=True))
model.add(TimeDistributed(Dense(units=13,activation="softmax")))

In [40]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])

In [42]:
model.fit(final_x_d,final_y_d,epochs=100,batch_size=150,validation_split=0.2)

Epoch 1/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 270s 815ms/step - accuracy: 0.9481 - loss: 0.1737 - val_accuracy: 0.9731 - val_loss: 0.0950
Epoch 2/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 263s 802ms/step - accuracy: 0.9695 - loss: 0.1137 - val_accuracy: 0.9827 - val_loss: 0.0582
Epoch 3/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 264s 804ms/step - accuracy: 0.9806 - loss: 0.0692 - val_accuracy: 0.9917 - val_loss: 0.0344
Epoch 4/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 254s 775ms/step - accuracy: 0.9894 - loss: 0.0426 - val_accuracy: 0.9945 - val_loss: 0.0232
Epoch 5/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 250s 764ms/step - accuracy: 0.9928 - loss: 0.0286 - val_accuracy: 0.9956 - val_loss: 0.0178
Epoch 6/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 262s 800ms/step - accuracy: 0.9947 - loss: 0.0210 - val_accuracy: 0.9961 - val_loss: 0.0148
Epoch 7/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 240s 730ms/step - accuracy: 0.9959 - loss: 0.0164 - val_accuracy: 0.9963 - val_loss: 0.0129
Epoch 8/100
327/327 ━━━━━━━━━━━━━━━━━━━━ 238s 728ms/step - accuracy: 

In [43]:
te=["Summer is one of the four temperate seasons, occurring after spring and before autumn."]

In [44]:
te

['Summer is one of the four temperate seasons, occurring after spring and before autumn.']

In [45]:
tk_x.texts_to_sequences(te)

[[810, 10, 40, 4, 1, 285, 23730, 6009, 5174, 99, 866, 5, 104, 4544]]

In [46]:
tk_x.sequences_to_texts(tk_x.texts_to_sequences(te))

['summer is one of the four temperate seasons occurring after spring and before autumn']

In [47]:
test=pad_sequences(tk_x.texts_to_sequences(te),maxlen=271,padding='post')

In [48]:
model.predict(test).shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step


(1, 271, 13)

In [49]:
import numpy as np

In [50]:
np.argmax(model.predict(test)[0],axis=1)[np.argmax(model.predict(test)[0],axis=1)!=0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


array([ 1,  2, 11,  4,  5, 11,  6,  1,  2,  4,  1,  9,  4,  1],
      dtype=int64)

In [51]:
tk_y.sequences_to_texts([[1,1,1,2,2,1,1,5,3,3]])

['noun noun noun verb verb noun noun det . .']

In [53]:
import pickle

In [54]:
pickle.dump(model,open("pos_model.pkl","wb"))

In [55]:
pickle.dump(tk_x,open("tk_x.pkl","wb"))
pickle.dump(tk_y,open("tk_y.pkl","wb"))

In [56]:
t1 = [" ".join(x[0])]

In [57]:
tk_x.texts_to_sequences(t1)

[[5594, 21728, 9068, 105, 158, 53, 1748, 1, 344, 18, 7, 13148, 880, 4000]]

In [58]:
tk_x.sequences_to_texts(tk_x.texts_to_sequences(t1))

['pierre vinken 61 years old will join the board as a nonexecutive director 29']

In [59]:
test_2=pad_sequences(tk_x.texts_to_sequences(t1),maxlen=271,padding='post')

In [60]:
model.predict(test_2).shape

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


(1, 271, 13)

In [61]:
np.argmax(model.predict(test_2)[0],axis=1)[np.argmax(model.predict(test_2)[0],axis=1)!=0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


array([ 1,  1, 11,  1,  6,  2,  2,  5,  1,  7,  5,  6,  1, 11],
      dtype=int64)

In [62]:
tk_y.sequences_to_texts([[1, 1, 1, 1, 4, 2, 2, 5, 1, 4, 5, 1, 1, 1]])

['noun noun noun noun adp verb verb det noun adp det noun noun noun']